In [1]:
from imutils import face_utils
import math
import dlib
import cv2
import os
import glob as gb
from PIL import Image, ImageDraw
from IPython import display
import torch
from torch import nn
import numpy as np
from PIL import Image, ImageDraw

/home/s439476@ucstaff.win.canberra.edu.au/anaconda3/envs/Dlib_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [3]:
#loading image
def Loading():
    samm_path ='../SAMM/Raw_Data/SAMM/*'
    casme_path = '../CASME-II/Cropped/*'
    smic_path = '../SMIC/SMIC_all_cropped/HS/*'
    #get subject
    sub_list = gb.glob(samm_path)
    sub = sub_list[0]
    sub_name = sub.split('/')[-1]
    # get video
    video_list = gb.glob(os.path.join(sub,'*'))
    if len(video_list) > 0:
        video = video_list[0]
    else:
        video = video_list[-1]
    video_name = video.split('/')[-1]
    # get frame
    frame_list = gb.glob(os.path.join(video,'*.jpg'))
    frame = frame_list[:2]
    return sub_name, video_name, frame_list  

In [4]:
#preprocess image
def ProImg(frame_name):
    img = cv2.imread(frame_name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face = detector(gray)[0]
    landmarks = predictor(gray, face)
    
    return img, landmarks

In [5]:
 # getting the distance between the most left point to eyes, width and height of the first image
def Reference(frame_dir): 
    
    #preprocess image
    img, landmarks = ProImg(frame_dir)
    
    # Determine the max and min points around the face
    x_min = np.min([landmarks.part(n).x for n in range(68)])
    x_max = np.max([landmarks.part(n).x for n in range(68)])
    y_top = np.min([landmarks.part(n).y for n in range(68)])
    y_max = np.max([landmarks.part(n).y for n in range(68)])
#     eye_left = (landmarks.part(36).x, landmarks.part(36).y)
    nose = (landmarks.part(30).x, landmarks.part(30).y)
    
    #Determine cropping bounding box
    width = abs(x_max-x_min)
    height = abs(y_max-y_top)
    y_min = y_top-abs(width-height)
    location = (nose[0] - x_min, nose[1]-y_min)

#     # Draw the rectangle on the image
#     cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)

#     # Draw the line on the image
#     cv2.line(img, (x_min, eye_left[1]),eye_left, (0, 255, 0), 2)
#     cv2.line(img, (eye_left[0], y_min),eye_left, (0, 255, 0), 2)


#     # Display the image
#     cv2.imshow('Image with Rectangle', img)
    
#     # Wait for a key press and then close the window
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return width, height, location

In [6]:
def normalize(frame_dir, width, height, location):

    Determine the eye's position 
    eye_left = (landmarks.part(36).x, landmarks.part(36).y)
    nose = (landmarks.part(30).x, landmarks.part(30).y)
    
    # Determine the bounding box around the face
    x_min = nose[0]-location[0]
    x_max = x_min+width
    y_min = nose[1]-location[1]
    y_max = y_min+width
    
#         # Draw the rectangle on the image
#     cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
#         # Display the image
#     cv2.imshow('Image with Rectangle', img)
    
#     # Wait for a key press and then close the window
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
    # Crop the image based on the bounding box
    cropped = img[y_min:y_max, x_min:x_max]
    
    return cropped

In [11]:
#Geting location of the first frame
sub_name, video_name, framelist = Loading() 
save_path = 'SAMM_Cropped_Dlib'
width, height, location = Reference(framelist[0])
ref_img, ref_landmarks = ProImg(framelist[0])

#normalize the rest frames:
for frame in framelist:
    #create save path
    frame_name = frame.split('/')[-1]
    frame_save_path = os.path.join(save_path, sub_name, video_name+'_ByLWM')
    folder = os.path.exists(frame_save_path)
    if not folder:
        os.makedirs(frame_save_path)
    frame_save_path = os.path.join(frame_save_path, frame_name)
    
    #normalize and crop rest images
    cropped_face = normalize(frame, width, height, location)
    cv2.imwrite(frame_save_path, cropped_face)

src_pts: <_dlib_pybind11.full_object_detection object at 0x7f49f85d4e30>


AxisError: axis 0 is out of bounds for array of dimension 0